
This appendix presents the pseudocode for both the classification and optimization algorithms defined in this study. 

The optimization algorithm includes the classification algorithm. Therefore to simplify this section only the optimization algorithm will be explained in detail.

###Optimization Algorithm
The optimization algorithm consists of three main subroutines: 
1. an **Initialization** function that reads in the pressure and wave data,
2. **Classification** in which CPs are assigned to classes and 
3. **Optimization** in which the classes are derived. Procedures of strategic importance are described as follows:



**INITIALIZATION**

Reads in pressure data,<br>
wave data and initial random definitions of CP classes.<br> Converts pressure values to anomalies.<br>

INPUT:

   Pressure data file,<br>
   wave data file,<br>
   CP class file,<br>
   Files containing average and standard deviations of pressure values,<br>
   number of grid points in the y-direction $Y$,<br>
   number of grid points in the x-direction $X$,<br>
   number of CP classes ($n$),<br>
   length of record $T$.
   
OUPTUT:

   Anomaly data matrix ($P \in \mathbb{R}^{T \times Y \times X}$),<br>
   Wave height vector ($W \in \mathbb{R}^{T}$),<br>
   CP Class matrix ($C \in \mathbb{R}^{n \times Y \times X}$).
    
$\underline{Variables}$:

 $C \in \mathbb{R}^{n\times Y \times X}$<br>
 $P \in \mathbb{R}^{T\times Y \times X}$<br>
 $W \in \mathbb{R}^{\period}$<br>
 $\mathbf{AVG} \in \mathbb{R}^{J \times Y \times X}$<br>
 $\mathbf{STD} \in \mathbb{R}^{J \times Y \times X}$ $J$ is the length of the year<br>
 $\mathbf{BA} \in \mathbb{R}^{Y\times X}$<br>
 $\mathbf{BS} \in \mathbb{R}^{Y\times X}$<br>
 Temp vectors of length $Y\times X$ to store values.

 $\mathbf{OPEN(File:}$ average pressure data file as $\mathbf{A})$<br>
 $\mathbf{OPEN(File:}$ standard deviation pressure data file as $\mathbf{S})$<br>
 For $i =1 \to J$  Read and store average and standard deviations of pressure data<br>
     &nbsp;$dummy \gets \mathbf{READ(A)}$ Dummy variable of time step in file<br>
     &nbsp;$dummy \gets \mathbf{READ(S)}$<br>
     &nbsp;$(\mathbf{BA}(k), k=1\to Y\times X) \gets \mathbf{READ(A},float)$<br>
     &nbsp;$(\mathbf{BS}(k), k=1\to Y\times X) \gets \mathbf{READ(S},float)$<br>
     &nbsp;$LL \gets 1$<br>
     &nbsp;For $k =1 \to \lat$<br>
         &nbsp;&nbsp;For$l =1 \to \lon$<br>
             &nbsp;&nbsp;&nbsp;$\mathbf{AVG}(k,l) \gets BA(LL)$<br>
             &nbsp;&nbsp;&nbsp;$\mathbf{STD}(k,l) \gets BS(LL)$<br>
             &nbsp;&nbsp;$LL \gets LL +1$<br>
         &nbsp;&nbsp;EndFor<br>
      &nbsp;EndFor<br>
EndFor

 $\mathbf{OPEN(File:}$ CP classes file as $\mathbf{CP})$<br>
For $i =1 \to n$ Read and store CP classes<br>
&nbsp;$dummy \gets \mathbf{READ(CP)}$ Dummy variable of class number in file<br>
$(\mathbf{BA}(k), k=1\to Y\times X) \gets \mathbf{READ(CP,}int)$<br>
&nbsp;$LL \gets 1$<br>
&nbsp;&nbsp;For $k =1 \to Y$<br>
&nbsp;&nbsp;For $l =1 \to X$<br>
&nbsp;&nbsp;&nbsp;$C(k,l) \gets BA(LL)$<br>
&nbsp;&nbsp;&nbsp;$LL \gets LL +1$<br>
&nbsp;&nbsp;EndFor<br>
&nbsp;EndFor<br>
EndFor

$\mathbf{OPEN(File:}$ Pressure data file as $\mathbf{Pr})$<br>
 $\mathbf{OPEN(File:}$ Wave data file as $\mathbf{Wa})$<br>
For $i =1 \to T$ Read and store pressure and wave data
&nbsp;$dummy \gets \mathbf{READ(Pr)}$ Dummy variable of time step in file<br>
&nbsp;$(\mathbf{BA}(k), k=1\to Y\times X) \gets \mathbf{READ(Pr},float)$<br>
&nbsp;$\waves \gets \mathbf{READ(Wa},float)$<br>
&nbsp;$LL \gets 1$<br>
&nbsp;For $k =1 \to \lat$<br>
&nbsp;&nbsp;For $l =1 \to X$<br>
&nbsp;&nbsp;&nbsp;$P(k,l) \gets BA(LL)$<br>
&nbsp;&nbsp;&nbsp;$LL \gets LL +1$<br>
&nbsp;&nbsp;EndFor<br>
&nbsp;EndFor<br>
EndFor<br>
 Reading data complete now convert pressure data to anomalies<br>
For $i=1 \to T$<br>
&nbsp;$J \gets JulianDate(i)$ Convert loop variable $i$ to a Julian date<br>
&nbsp;For $k=1 \to Y$<br>
&nbsp;&nbsp;For $l=1 \to X$<br>
&nbsp;&nbsp;&nbsp;State $P(i,k,l) \gets \dfrac{\pressure(i,k,l) - \mathbf{AVG}(J,k,l)}{\mathbf{STD}(J,k,l)}$<br>
&nbsp;&nbsp;EndFor<br>
&nbsp;EndFor<br>
EndFor 

**CLASSIFICATION**

INPUT:

   number of CP classes ($n$),<br>
   Anomaly data matrix ($P \in \mathbb{R}^{T \times Y \times X}$),<br>
   CP Class matrix ($C \in \mathbb{R}^{n \times Y \times X}$),<br>
   $C$ contains the definitions of the CP classes derived in the optimization<br>
    number of grid points in the y-direction $Y$,<br>
    number of grid points in the x-direction $X$,<br>
    length of record $T$
    
OUTPUT:

A vector $c \in \mathbb{R}^{T}$ containing CP class labels for all time steps in $T$.

$\underline{Variables}$:<br>
$pex \in \mathbb{R}^{4}$ user defined exponent values for fuzzy numbers $1,\ldots, 4$<br>
 $N \in \mathbb{R}^{4}$ vector to store counts of different fuzzy numbers $1,\ldots, 4$<br>
$\mu \in \mathbb{R}^{4}$ vector to store membership grades<br>
Set all values of $N$ and $\mu$ to 0<br>
$DOF = 1$<br>
$Class = 1$<br>
$Max = 0$<br>
For i=1 $\to T$<br>
&nbsp;For j=1 $\to n$<br>
&nbsp;&nbsp; For k = 1$\to Y$<br>
&nbsp;&nbsp;&nbsp;For l = 1 $\to X$<br>
&nbsp;&nbsp;&nbsp;&nbsp; $Fuzzy Number$ $\gets$ $c(j,k,l)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$Anomaly$ $\gets$ $P(i,k,l)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$grade \gets Membership(Fuzzy Number, Anomaly)$<br>
&nbsp;&nbsp;&nbsp;Calculate the membership grade for anomaly value based on the fuzzy number<br>
&nbsp;&nbsp;&nbsp;&nbsp; $\mu(Fuzzy Number) \gets \mu(Fuzzy Number) + grade^{pex(Fuzzy Number)}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$N(Fuzzy Number) \gets N(Fuzzy Number) +1$<br>
&nbsp;&nbsp;&nbsp;EndFor<br>
&nbsp;&nbsp;EndFor<br>
&nbsp; For$m = 1: 4$<br>
&nbsp;&nbsp;$DOF \gets DOF \times \left(\frac{\mu(m)}{N(m)}\right)^{1/pex(m)}$<br>
&nbsp;EndFor<br>
&nbsp;If $DOF>Max$<br>
&nbsp;&nbsp;$Class \gets  i$<br>
&nbsp;&nbsp; $Max \gets DOF$<br>
&nbsp;EndIf<br>
&nbsp;EndFor<br>
&nbsp;$c(T) \gets Class$<br>
&nbsp;Set all values of $N, \mu$ and  $Max$ to 0.<br>
&nbsp;Set $DOF$ and $Class = 1$<br>
EndFor

**MEMBERSHIP**

This procedure assigns a membership grade $\in [0,1]$ to the anomaly at a grid point based on its value and the fuzzy number at the same grid point. It is a function that is called from the **CLASSIFICATION**
function.

INPUT:

Fuzzy Number ($f$)  and the Anomaly value ($a$)

OUTPUT:

Membership grade $\mu$

&nbsp;$\mathbf{Function} Triangular(a,x,y,z)$<br>
&nbsp;Calculate the membership grade using a triangular function<br>
&nbsp;$\mu = \left\{\begin{array}{lc}
\dfrac{a-x}{y-x} & if x\leq a \leq x\\
\dfrac{a-z}{y-z} & if y < a \leq z\\
0 & else 
\end{array}
\right.$<br>

&nbsp;Return $\mu$

&nbsp;$\mathbf{EndFunction}$

If $f == 1$<br>
&nbsp; $\mu \gets Triangular(a, -\infty, -3, 0)$<br>
ElsIf $f == 2$<br>
&nbsp; $\mu \gets Triangular(a, -4, -0.85, 4)$<br>
ElsIf $f == 3$<br>
&nbsp; $\mu \gets Triangular(a, 0.25, 0.85, 4)$<br>
ElsIf $f == 4$<br>
&nbsp; $\mu \gets Triangular(a, 0, 3, +\infty)$<br>
EndIf

**OPTIMIZATION**

This optimization procedure uses simulated annealing to optimize the shape of the CP classes.

INPUT:

Number of CPs $n$,<br>
Wave vector $W \in \mathbb{R}^{T}$,<br>
Anomaly matrix $P \in \mathbb{R}^{T \times Y \times X}$,<br>
CP classes $C \in \mathbb{R}^{n \times Y \times X}$,<br>
grid points in the y-direction $Y$,<br>
grid points in the x-direction $X$,<br>
Time period $T$.<br>

OUTPUT:

An optimized CP class matrix $C \in \mathbb{R}^{n \times Y \times X}$.<br>
 $c \gets Classification(n, P, C, Y, X, T)$
Perform initial classification<br>
$\mathbf{Ob1}$, $\mathbf{Ob2}$, $\mathbf{Ob3}$ $\gets CalcObjectiveFuncs(c,W)$<br>
Calculate initial objective functions<br>
$\mathbf{Temp} \gets 7.85*n$ Randomly set initial annealing temperature<br>
For $kt = 1 \to 150$ Start annealing<br>
&nbsp; For $i =1 \to 5000$<br>
&nbsp;&nbsp; $\mathbf{R} \gets Random(n)$ Randomly select a CP class<br>
&nbsp;&nbsp; $\mathbf{IX} \gets Random(X)$ Randomly select x-location<br>
&nbsp;&nbsp; $\mathbf{IY} \gets Random(Y)$ Randomly select y-location<br>
&nbsp;&nbsp; $C(R,IY,IX) \gets f \in [1,\ldots,4], f \neq C(R,IY,IX)$<br>
&nbsp;&nbsp;Change fuzzy number at location $(IX,IY)$ for class $R$<br>
&nbsp;&nbsp;$c2 \gets Classification(n,P,C,Y,X,T)$<br>
&nbsp;&nbsp; Perform new classification with changed CP class<br>
&nbsp;&nbsp; $\mathbf{Ub1}$, $\mathbf{Ub2}$, $\mathbf{Ub3}$ $\gets CalcObjectiveFuncs(c2,W)$
&nbsp;&nbsp; $\mathbf{O} \gets \alpha\mathbf{Ob1} + \beta\mathbf{Ob2} + \gamma\mathbf{Ob3}$<br>
&nbsp;&nbsp; $\mathbf{U} \gets \alpha\mathbf{Ub1} + \beta\mathbf{Ub2} + \gamma\mathbf{Ub3}$<br>
&nbsp;&nbsp;Calculate old and new objective functions. Weights are user defined.<br>
&nbsp;&nbsp; $\mathbf{O} \gets w - \mathbf{O}$<br>
&nbsp;&nbsp; $\mathbf{U} \gets w - \mathbf{U}$<br>
&nbsp;&nbsp;Change maximization scheme to minimizing<br>
&nbsp;&nbsp;If $\mathbf{U} < \mathbf{O}$<br>
&nbsp;&nbsp;&nbsp; Accept change<br>
&nbsp;&nbsp;Else<br>
&nbsp;&nbsp;&nbsp; Accept change with probability<br>
$\exp\left(\dfrac{\mathbf{O}-\mathbf{U}}{\mathbf{Temp}}\right)$<br>
&nbsp;&nbsp;EndIf<br>
&nbsp;&nbsp;If Change is accepted<br>
&nbsp;&nbsp; $\mathbf{O1}, \mathbf{O2}, \mathbf{O3} \gets \mathbf{U1}, \mathbf{U2}, \mathbf{U3}$<br>
&nbsp;&nbsp;EndIf<br>
&nbsp;&nbsp; Reduce annealing temperature.<br>
&nbsp;EndFor<br>
EndFor

**CalcObjectiveFuncs**

This procedures calculates the objective functions required for the optimization. It is called in the optimization procedure.

INPUT:

Classification $c \in \mathbb{R}^{T}$,<br>
Wave data $W \in \mathbb{T}$,<br>
Period $T$.

OUTPUT:

Objective functions $\mathbf{O1}, \mathbf{O2}$ and $\mathbf{O3}$.<br>

$\underline{Variables}$:

$\mathbf{h_{CP}} \in \mathbb{R}^{n} \gets 0$ Frequency of CP class<br>
$\mathbf{hs_{CP}} \in \mathbb{R}^{n} \gets 0$ Average wave height of CP class<br>
\$\mathbf{hs} \gets 0$ Average wave height<br>
$\mathbf{pr_{3.5}} \in \mathbb{R}^n \gets 0$ Probability of wave height exceeding 3.5~m for a given CP<br>
$\mathbf{pe_{3.5}} \gets 0$ Probability of wave height exceeding 3.5~m<br>
$\mathbf{pr_{2.5}} \in \mathbb{R}^n \gets 0$ Probability of wave height exceeding 2.5~m for a given CP<br>
$\mathbf{pe_{2.5}} \gets 0$ Probability of wave height exceeding 2.5~m<br>

For $i=1 \to T$<br>
&nbsp;\State $cp \gets c(i)$<br>
&nbsp;$\mathbf{h_{CP}}(cp) \gets \mathbf{h_{CP}}(cp) +1$<br>
&nbsp;$\mathbf{hs_{CP}}(cp) \gets \mathbf{hs_{CP}}(cp) +\waves(i)$<br>
&nbsp; $\mathbf{hs} \gets \mathbf{hs} +W(i)$
&nbsp;\If{$W(i)\geq2.5$}<br>
&nbsp;&nbsp; $\mathbf{pr_{2.5}}(cp) \gets \mathbf{pr_{2.5}}(cp)+1$<br>
&nbsp;&nbsp;$\mathbf{pe_{2.5}} \gets \mathbf{pe_{2.5}} +1$<br>
&nbsp;EndIf<br>
&nbsp;If $W(i)\geq3.5$<br>
&nbsp;&nbsp; $\mathbf{pr_{3.5}}(cp) \gets \mathbf{pr_{3.5}}(cp)+1$<br>
&nbsp;&nbsp; $\mathbf{pe_{3.5}} \gets \mathbf{pe_{3.5}} +1$<br>
&nbsp;EndIf<br>
EndFor

$\mathbf{hs} \gets \mathbf{hs}/T$<br>
$\mathbf{pe_{2.5}} \gets \mathbf{pe_{2.5}}/T$<br>
$\mathbf{pe_{3.5}} \gets \mathbf{pe_{3.5}}/T$<br>
For $i=1 \to n$<br>
&nbsp; $\mathbf{hs_{CP}}(i) \gets \mathbf{hs_{CP}}(i)/\mathbf{h_{CP}}(i)$<br>
&nbsp;$\mathbf{O1} \gets \mathbf{O1} + \mathbf{hs_{CP}}(i)/\mathbf{hs}$<br>
&nbsp;$\mathbf{pr_{2.5}}(i) \gets \mathbf{pr_{2.5}}(i)/\mathbf{h_{CP}}(i)$<br>
&nbsp;$\mathbf{O2} \gets \mathbf{O2} + (\mathbf{pr_{2.5}}(i) - \mathbf{pe_{2.5}})^2$<br>
&nbsp;$\mathbf{pr_{3.5}}(i) \gets \mathbf{pr_{3.5}}(i)\mathbf{h_{CP}}(i)$<br>
&nbsp;$\mathbf{O3} \gets \mathbf{O3} + (\mathbf{pr_{3.5}}(i) - \mathbf{pe_{3.5}})^2$<br>
EndFor


